## Notebook setup


In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import pandas as pd
import base64
import pytz

from IPython.display import HTML

import openbb_terminal.config_terminal as cfg
from openbb_terminal.helper_funcs import get_user_timezone
from openbb_terminal import OpenBBFigure, theme
from openbb_terminal.helper_classes import TerminalStyle


warnings.filterwarnings("ignore")

In [ ]:
# Suppressing sdk logs
from openbb_terminal.core.session.current_system import set_system_variable

set_system_variable("LOGGING_SUPPRESS", True)

# Import the OpenBB SDK
from openbb_terminal.sdk import openbb, widgets

In [ ]:
from openbb_terminal.core.session.current_user import get_current_user

user = get_current_user()

cfg.theme = TerminalStyle(
    user.preferences.MPL_STYLE,
    user.preferences.PMF_STYLE,
    user.preferences.RICH_STYLE,
)
theme.apply_style("light")

In [ ]:
stylesheet = widgets.html_report_stylesheet()

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
report_name = "economy_report"

In [ ]:
author = "OpenBB"
report_title = f"Economy report"
user_time_zone = pytz.timezone(get_user_timezone())
report_date = pd.Timestamp.now(tz=user_time_zone).strftime("%d %B, %Y")
report_time = pd.Timestamp.now(tz=user_time_zone).strftime("%H:%M")
report_timezone = pd.Timestamp.now(tz=user_time_zone).tzinfo
report_title, report_date, report_time, report_timezone

## Data


Summary


In [ ]:
overview = openbb.economy.overview()
overview.set_index(list(overview.columns[0]), inplace=True)
overview = OpenBBFigure.to_table(overview, print_index=True, width=600)

futures = openbb.economy.futures()
futures.set_index(list(futures.columns[0]), inplace=True)
futures["%Chg"] = futures["%Chg"].apply(lambda x: float(f"{x:.2f}"))
futures_fig = OpenBBFigure.to_table(futures, print_index=True)

In [ ]:
try:
    news = openbb.news("economy")
    news.columns = news.columns.str.title()
    news = news.rename(
        columns={"Date": "Published", "Description": "Title", "Url": "Link"}
    )
    news["Title"] = news.apply(
        lambda x: f'<a href="{x["Link"]}">{x["Title"]}</a>', axis=1
    )
    news = news[["Title", "Published"]]
    news["Published"] = pd.to_datetime(news["Published"]).dt.strftime("%Y-%m-%d %H:%M")
    news = news.set_index("Published")
except:
    news = pd.DataFrame()

Events


In [ ]:
events_1 = openbb.economy.events(
    countries="United States",
    start_date=datetime.datetime.now().strftime("%Y-%m-%d"),
)
if not events_1.empty:
    events_1 = OpenBBFigure.to_table(
        events_1.set_index("Date"), print_index=True, height=200
    )

events_2 = openbb.economy.events(
    countries=[
        "Germany",
        "Spain",
        "Italy",
        "United Kingdom",
        "France",
        "Portugal",
        "Greece",
    ],
    start_date=datetime.datetime.now().strftime("%Y-%m-%d"),
)
if not events_2.empty:
    events_2 = OpenBBFigure.to_table(events_2.set_index("Date"), print_index=True)

events_3 = openbb.economy.events(
    countries="China",
    start_date=datetime.datetime.now().strftime("%Y-%m-%d"),
)
if not events_3.empty:
    events_3 = OpenBBFigure.to_table(events_3.set_index("Date"), print_index=True)

Yields


In [ ]:
ycrv = openbb.fixedincome.ycrv(return_date=True)

if (isinstance(ycrv, pd.DataFrame) and not ycrv.empty) or (
    not isinstance(ycrv, pd.DataFrame) and ycrv
):
    ycrv_data_1, _ = ycrv
    ycrv_country_1 = "United states"

    ycrv_chart_1 = openbb.fixedincome.ycrv_chart(external_axes=True)

    if ycrv_chart_1:
        ycrv_chart_1 = ycrv_chart_1.to_html()

    if not ycrv_data_1.empty:
        ycrv_data_1.drop(["Maturity"], axis=1, inplace=True)
        ycrv_data_1.index = [
            "1M",
            "3M",
            "6M",
            "1Y",
            "2Y",
            "3Y",
            "5Y",
            "7Y",
            "10Y",
            "20Y",
            "30Y",
        ]
else:
    ycrv = None

In [ ]:
subplots = OpenBBFigure.create_subplots(
    2,
    2,
    vertical_spacing=0.15,
    row_heights=[0.5, 0.5],
    subplot_titles=("UST 1m-1y", "UST 10y-30y", "UST 1y-10y"),
    specs=[[{"type": "xy"}, {"type": "xy"}], [{"colspan": 2}, None]],
)
openbb.economy.treasury_chart(
    maturities=["1m", "3m", "6m", "1y"],
    start_date="2020-01-01",
    external_axes=True,
).to_subplot(subplots, 1, 1)

openbb.economy.treasury_chart(
    maturities=["10y", "20y", "30y"],
    start_date="2020-01-01",
    external_axes=True,
).to_subplot(subplots, 1, 2)

openbb.economy.treasury_chart(
    maturities=["1y","5y","7y", "10y"],
    start_date="2020-01-01",
    external_axes=True,
).to_subplot(subplots, 2, 1)

subplots.update_traces(legend="legend", row=1, col=1)
subplots.update_traces(legend="legend2", row=1, col=2)
subplots.update_traces(legend="legend3", row=2, col=1)
defaults = dict(
    xanchor="left",
    yanchor="top",
    xref="paper",
    yref="paper",
    bgcolor="rgba(0, 0, 0, 0)",
)
subplots.update_layout(
    legend=dict(x=0.01, y=0.99, **defaults),
    legend2=dict(x=0.562, y=0.99, **defaults),
    legend3=dict(x=0.01, y=0.41, **defaults),
)


ust = subplots.to_html()

Equity Market


In [ ]:
valuation = OpenBBFigure.to_table(openbb.economy.valuation(), print_index=False)
performance = OpenBBFigure.to_table(openbb.economy.performance(), print_index=False)

In [ ]:
fig = OpenBBFigure(yaxis_title="Cumulative returns (%)")
data = openbb.economy.index(
    indices=["sp500", "dow_djus", "nasdaq100", "russell3000", "cboe_vix"],
    start_date="2000-03-09",
)

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
for col in data.columns:
    fig.add_scatter(
        x=data.index,
        y=data[col] * 100,
        name=col,
    )

eqty_0 = fig.update_layout(margin=dict(l=20, r=60), legend=dict(x=1.1)).to_html()

In [ ]:
fig = OpenBBFigure(yaxis_title="Cumulative returns (%)")
data = openbb.economy.index(
    indices=["ftse100", "de_dax40", "fr_cac40", "it_mib40", "es_ibex35"],
    start_date="2000-01-01",
)

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
for col in data.columns:
    fig.add_scatter(
        x=data.index,
        y=data[col] * 100,
        name=col,
    )

eqty_1 = fig.update_layout(margin=dict(l=20, r=60), legend=dict(x=1.1)).to_html()

In [ ]:
fig = OpenBBFigure(yaxis_title="Cumulative returns (%)")
data = openbb.economy.index(
    indices=["jp_n225", "cn_csi300", "hk_china", "au_asx50"], start_date="2014-01-01"
)

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
for col in data.columns:
    fig.add_scatter(
        x=data.index,
        y=data[col] * 100,
        name=col,
    )

eqty_2 = fig.update_layout(margin=dict(l=0, r=60), legend=dict(x=1.1)).to_html()

In [ ]:
fig = OpenBBFigure(yaxis_title="Cumulative returns (%)")
data = openbb.economy.index(
    indices=["in_nse50", "br_bvsp", "za_jo", "mx_ipc", "ru_moex"],
    start_date="2013-01-01",
)

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
for col in data.columns:
    fig.add_scatter(
        x=data.index,
        y=data[col] * 100,
        name=col,
    )

eqty_3 = fig.update_layout(margin=dict(l=20, r=60), legend=dict(x=1.1)).to_html()

In [ ]:
fig = OpenBBFigure(yaxis_title="Cumulative returns (%)")
data = openbb.economy.index(indices=["arca_xoi", "arca_pse"], start_date="2000-01-01")

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
for col in data.columns:
    fig.add_scatter(
        x=data.index,
        y=data[col] * 100,
        name=col,
    )

eqty_4 = fig.update_layout(margin=dict(l=20, r=60), legend=dict(x=1.1)).to_html()

In [ ]:
bigmac_res = openbb.economy.bigmac()

if (isinstance(bigmac_res, pd.DataFrame) and not bigmac_res.empty) or (
    not isinstance(bigmac_res, pd.DataFrame) and bigmac_res
):
    bigmac = openbb.economy.bigmac_chart(
        country_codes=[
            "USA",
            "CAN",
            "ARG",
            "PRT",
            "NLD",
            "FRA",
            "UAE",
            "LKA",
            "VNM",
            "RUS",
            "SWE",
            "GBR",
        ],
        external_axes=True,
    )
    if bigmac:
        bigmac.set_title("")
        bigmac = bigmac.update_layout(
            margin=dict(l=0, r=60), legend=dict(x=1.1)
        ).to_html()

else:
    bigmac_res = pd.DataFrame()

In [ ]:
subplots = OpenBBFigure.create_subplots(1, 1, specs=[[{"secondary_y": True}]])

openbb.economy.macro_chart(
    parameters=["CONF", "IP"], start_date="2010-01-01", external_axes=True
).to_subplot(subplots, 1, 1, secondary_y=False)
subplots.update_yaxes(title_text="Confidence and IP", secondary_y=False)


df, df_dict, _ = openbb.economy.macro(parameters=["CPI"], start_date="2010-01-01")
cpi_chg = df.pct_change(1) * 100
subplots.add_scatter(
    x=cpi_chg.index,
    y=cpi_chg.T.values[0],
    name="CPI (% change)",
    mode="lines",
    line=dict(color="red", dash="dash"),
    secondary_y=True,
    yaxis="y2",
)
subplots.update_yaxes(
    title_text="CPI (% change)",
    secondary_y=True,
    range=[-2, 2],
    side="left",
    overlaying="y",
)
subplots.horizontal_legend()

macro_0 = subplots.to_html()

In [ ]:
fred = openbb.economy.fred(series_ids=["T10Y3M"], start_date="1980-01-01")

if (isinstance(fred, pd.DataFrame) and not fred.empty) or (
    not isinstance(fred, pd.DataFrame) and fred
):
    df, df_dict = fred

    fig = OpenBBFigure.create_subplots(1, 1, specs=[[{"secondary_y": True}]])
    fig.add_scatter(
        x=df.index,
        y=df["T10Y3M"],
        name=df_dict["T10Y3M"]["title"],
        line=dict(color="red", dash="dash"),
        secondary_y=False,
    )
    openbb.economy.fred_chart(
        series_ids=["A191RP1A027NBEA"],
        start_date="1980-01-01",
        external_axes=True,
    ).to_subplot(fig, 1, 1, secondary_y=True)
    fig.update_yaxes(secondary_y=True, side="left", overlaying="y")
    fig.set_title("")
    fig.horizontal_legend()

    fred_0 = fig.to_html()
else:
    fred = None

## Render the report template to a file


In [ ]:
body = ""


body += widgets.header(
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>ECONOMY REPORT: Leverage multiple sources to create custom reports.</b>",
    plotly_js=True,
)

body += widgets.tablinks(
    [
        "SUMMARY",
        "Events",
        "Yields",
        "Equity Market",
        "Macro Data",
    ]
)

# Summary
htmlcode = widgets.row(
    [widgets.h(3, "Global markets (source: WSJ)") + overview.to_html()]
)

htmlcode += widgets.row(
    [widgets.h(3, "Commodities (source: WSJ)") + futures_fig.to_html()]
)

if not news.empty:
    htmlcode += widgets.row(
        [
            widgets.h(3, "Top news on 'economy' keyword (source: Feedparser)")
            + news.to_html(escape=False)
        ]
    )

body += widgets.add_tab("SUMMARY", htmlcode)

# Events
htmlcode = widgets.row([widgets.h(3, "Economic calendars (source: investing.com)")])
htmlcode += widgets.row([widgets.h(3, "United States") + events_1.to_html()])
htmlcode += widgets.row(
    [widgets.h(3, "Select European Countries") + events_2.to_html()]
)
htmlcode += widgets.row([widgets.h(3, "China") + events_3.to_html()])
body += widgets.add_tab("Events", htmlcode)

# Yields
htmlcode = widgets.row([widgets.h(3, "US treasuries (source: FED)") + ust])

if ycrv:
    htmlcode += widgets.row(
        [
            widgets.h(3, f"{ycrv_country_1} yield curve (source: FRED)")
            + (ycrv_chart_1 or "")
        ]
    )
    htmlcode += widgets.row([widgets.h(3, "") + ycrv_data_1.to_html()])

body += widgets.add_tab("Yields", htmlcode)

# Equity market
htmlcode = widgets.row(
    [widgets.h(3, "Valuation (source: Finviz)") + valuation.to_html()]
)
htmlcode += widgets.row(
    [widgets.h(3, "Performance (source: Finviz)") + performance.to_html()]
)
htmlcode += widgets.row([widgets.h(3, "US markets (source: YahooFinance)") + eqty_0])
htmlcode += widgets.row(
    [widgets.h(3, "European markets (source: YahooFinance)") + eqty_1]
)
htmlcode += widgets.row([widgets.h(3, "APAC markets (source: YahooFinance)") + eqty_2])
htmlcode += widgets.row(
    [widgets.h(3, "Emerging markets (source: YahooFinance)") + eqty_3]
)
htmlcode += widgets.row([widgets.h(3, "ARCA (source: YahooFinance)") + eqty_4])
body += widgets.add_tab("Equity Market", htmlcode)

# Macro
htmlcode = ""
if not bigmac_res.empty:
    htmlcode += widgets.row(
        [widgets.h(3, "Bigmac index (source: Nasdaq)") + (bigmac or "")]
    )
htmlcode += widgets.row([widgets.h(3, "Leading indicators (source: EconDB)") + macro_0])
if fred:
    htmlcode += widgets.row(
        [widgets.h(3, "Recession indicator (source: FRED)") + fred_0]
    )
body += widgets.add_tab("Macro Data", htmlcode)

body += widgets.tab_clickable_and_save_evt()
report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)